<a href="https://colab.research.google.com/github/aimbesi1/CSC_4850-MachineLearning-Project/blob/main/ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Default imports

%matplotlib inline

import numpy as np
import pandas as pd
import sklearn 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# Specific imports
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline

Using the housing dataset from Kaggle: https://www.kaggle.com/datasets/mirbektoktogaraev/madrid-real-estate-market

In [2]:
### Manually download it and upload to this istance data sample space
### Note DO NOT change these operations or all your answers will be incorrect

### Let's do some transformations and extra features on this.
df=pd.read_csv('houses_Madrid.csv', encoding='utf-8')

In [3]:
# Select/clean features here

# Do what needs to be done to the data so that the models can run

In [40]:
# Update X values later
df_short = df.iloc[:1000, :][['sq_mt_built', 'rent_price', 'buy_price']].dropna()
df_short['high_price'] = df_short['buy_price'].apply(lambda x: 1 if x > np.median(df_short['buy_price']) else 0)
X = df_short[['sq_mt_built', 'rent_price']]
# y = df_short[['buy_price']]
y = df_short['high_price'].values



In [16]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn import svm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

def init_models():
  dt = DecisionTreeRegressor(random_state=1234)
  perceptron = Perceptron(random_state=1234)
  nb = GaussianNB()
  log_reg = LogisticRegression(random_state=1234)
  lin_reg = LinearRegression()
  ridge = Ridge(alpha=0.5, fit_intercept=True, random_state=1234)
  lasso = Lasso(alpha=0.1, fit_intercept=True, random_state=1234)
  ef = ElasticNet(alpha=0.9, fit_intercept=True, random_state=1234)
  svm_linear = svm.SVC(kernel="linear", random_state=1234)
  svm_rbf = svm.SVC(kernel="rbf", random_state=1234)
  gbr = GradientBoostingRegressor(random_state=1234)
  mlp = MLPRegressor(random_state=1234)
  svr_poly = svm.SVR(kernel="poly")

  dict = {
      "dt": dt,
      "perceptron": perceptron,
      "nb": nb,
      "log_reg": log_reg,
      "lin_reg": lin_reg,
      "ridge": ridge,
      "lasso": lasso,
      "ef": ef,
      "svm_linear": svm_linear,
      "svm_rbf": svm_rbf,
      "gbr": gbr,
      "mlp": mlp,
  }

  return dict

In [57]:
def run_models(models, X_train, X_test, y_train, y_test, lc_dict, m_dict, b_dict):
  def get_metrics(model, X_train, y_train):
    # lc = learning_curve(estimator=model, X=X_train, y=y_train, cv=10, shuffle=True, random_state=1234)
    lc = -1
    cv = cross_validate(model, X_train, y_train, cv=10, scoring=("precision", "recall", "accuracy", "f1"), return_train_score=False, return_estimator=True)
    best_score_index = cv["test_accuracy"].argmax()
    best = cv["estimator"][best_score_index]
    return lc, cv, best


  for key in models.keys():
    lc, cv, best = None, None, None
    if key in {"svm_lin", "lin_reg", "ridge", "lasso", "ef"}:
      X_train_scaled = StandardScaler().fit_transform(X_train)
      X_test_scaled = StandardScaler().fit_transform(X_test)

      models[key].fit(X_train_scaled, y_train)
      predY = models[key].predict(X_test_scaled)
      lc, cv, best = get_metrics(models[key], X_train_scaled, y_train)
    else:
      models[key].fit(X_train, y_train)
      predY = models[key].predict(X_test)
      lc, cv, best = get_metrics(models[key], X_train, y_train)
    
    # lc_dict[key] = lc
    m_dict[key] = cv
    b_dict[key] = best

def test_models(models, X_train, X_test, y_train, y_test, lc_dict, bp_dict):
  for key in models.keys():
    lc, predY = None, None
    if key in {"svm_lin", "lin_reg", "ridge", "lasso", "ef"}:
      X_train_scaled = StandardScaler().fit_transform(X_train)
      X_test_scaled = StandardScaler().fit_transform(X_test)

      models[key].fit(X_train_scaled, y_train)
      predY = models[key].predict(X_test_scaled)
      lc = learning_curve(estimator=models[key], X=X_test_scaled, y=y_test, shuffle=True, random_state=1234)
    else:
      models[key].fit(X_train, y_train)
      predY = models[key].predict(X_test)
      lc = learning_curve(estimator=models[key], X=X_test, y=y_test, shuffle=True, random_state=1234)

    lc_dict[key] = lc
    bp_dict[key] = predY



  
  



# def run_models(models, X_train, y_train, X_test, y_test):

#   # Decision Tree
#   models["dt"].fit(X_train, y_train)
#   pred_Y1 = models["dt"].predict(X_test)
#   print(metrics.classification_report(y_test, pred_Y1))

#   # SVM Linear
#   f1_scaledX = StandardScaler().fit(X_train).transform(X_train)
#   f1_scaledX_test = StandardScaler().fit(X_test).transform(X_test)

#   models["svm_linear"].fit(f1_scaledX, y_train)
#   pred_Y1 = models["svm_linear"].predict(f1_scaledX_test)
#   print(metrics.classification_report(y_test, pred_Y1))

#   sizes1, train_scores1, valid_scores1 = learning_curve(estimator = svm_linear, X = f1_scaledX_test, y=y_test, shuffle = True, random_state = 1234)

#   # RBF
#   models["svm_rbf"].fit(X_train, y_train)
#   pred_Y3 = models["svm_rbf"].predict(X_test)
#   print(metrics.classification_report(y_test, pred_Y3))

#   sizes3, train_scores3, valid_scores3 = learning_curve(estimator = svm_rbf, X = X_test, y=y_test, shuffle = True, random_state = 1234)


 

def show_learning_curve(ax, lc_sizes, train_scores, valid_scores, title):
  train_mean = train_scores.mean(axis=1)
  valid_mean = valid_scores.mean(axis=0)
  ax.plot(lc_sizes, train_mean, label = "Training Error - Sample 1")
  ax.plot(lc_sizes, valid_mean, label = "Validation Error - Sample 1")
  ax.legend()
  ax.set_title(title)


# # Learning curves
# f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (15, 5), sharey=True)
# show_learning_curve(ax1, sizes1, train_scores1, valid_scores1, "Linear")
# show_learning_curve(ax2, sizes2, train_scores2, valid_scores2, "Poly")
# show_learning_curve(ax3, sizes3, train_scores3, valid_scores3, "RBF")

# acc_linear = metrics.accuracy_score(fold["y_test"], pred_Y1)
# acc_poly = metrics.accuracy_score(fold["y_test"], pred_Y2)
# acc_rbf = metrics.accuracy_score(fold["y_test"], pred_Y3)
# return {"linear": svm_linear, "poly": svm_poly, "rbf": svm_rbf}

# f1_results = run_models(fold1, svm_linear, svm_poly, svm_rbf)

In [56]:
##### Pipeline

''' 
For each sampling:
  Set up the models
  Initialize L: list of LCs (list of training scores + valid scores)
  Initialize M: list of metrics (predicted Y)
  

'''


def pipe(X, y):
  train_sizes = [0.50, 0.70, 0.80]
  # learning_curves and metrics are arrays of 3 dictionaries each.
  # Each dictionary will hold the learning curve and metrics data for each model in each fold. 
  learning_curves = np.array([{}, {}, {}])
  metrics = np.array([{}, {}, {}])
  models = {}
  best_models = np.array([{}, {}, {}])
  best_predictions = np.array([{}, {}, {}])

  for i in range(len(train_sizes)):
    models = init_models()
    rbf = {"rbf": models["svm_rbf"]}
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_sizes[i], random_state=1234)

    # Test with 1 model
    run_models(rbf, X_train, X_test, y_train, y_test, learning_curves[i], metrics[i], best_models[i])
    
    
    # print(learning_curves[i])
    
    print(metrics[i])

    print(best_models[i])

    # Stop at 1 loop with break
    # break

    # Now train the best models on the whole training set
    
    test_models(rbf, X_train, X_test, y_train, y_test, learning_curves[i], best_predictions[i])


  
  # Learning curves
  f, ax = plt.subplots(1, len(models), figsize = (15, 5), sharey=True)
  
  for i in range(len(models)):
    


  
  

    


# def pipe(X, y):
#   train_sizes = [0.50, 0.70, 0.80]
#   for size in train_sizes:
#     models = init_models()
#     rbf = models["svm_rbf"]
#     X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=size, random_state=1234)

#     kfold = KFold(n_splits=10, random_state=1234, shuffle=True)

#     for index, (train, test) in enumerate(kfold.split(X_train,y_train)):
#       X_train_folds = X_train[train]
#       y_train_folds = y_train[train]

#       X_test_folds = X_train[test]
#       y_test_folds = y_train[test]

#       run_models(models, X_train_folds, y_train_folds, X_test_folds, y_test_folds)

pipe(X, y)

{'rbf': {'fit_time': array([0.00369096, 0.00281453, 0.00386024, 0.00286913, 0.0027194 ,
       0.00304985, 0.00322318, 0.00277352, 0.00333881, 0.00289774]), 'score_time': array([0.00470757, 0.00449228, 0.00494075, 0.0043242 , 0.00432658,
       0.00541663, 0.00431514, 0.00580645, 0.00430274, 0.00419831]), 'estimator': [SVC(random_state=1234), SVC(random_state=1234), SVC(random_state=1234), SVC(random_state=1234), SVC(random_state=1234), SVC(random_state=1234), SVC(random_state=1234), SVC(random_state=1234), SVC(random_state=1234), SVC(random_state=1234)], 'test_precision': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), 'test_recall': array([1.        , 0.95833333, 1.        , 0.95833333, 0.95833333,
       1.        , 0.91666667, 0.91666667, 1.        , 1.        ]), 'test_accuracy': array([1.  , 0.98, 1.  , 0.98, 0.98, 1.  , 0.96, 0.96, 1.  , 1.  ]), 'test_f1': array([1.        , 0.9787234 , 1.        , 0.9787234 , 0.9787234 ,
       1.        , 0.95652174, 0.95652174, 1.        , 1

In [7]:
def get_metrics(model):
  pred_Y = model.predict(X_test)
  acc = metrics.accuracy_score(y_test, pred_Y)
  precision = metrics.precision_score(y_test, pred_Y)
  recall = metrics.recall_score(y_test, pred_Y)
  f1_score = metrics.f1_score(y_test, pred_Y)
  rmse = np.sqrt(metrics.mean_squared_error(y_test, pred_Y))
  return [acc, precision, recall, f1_score, rmse]

# table = pd.DataFrame()
# table["Metric"] = ["Accuracy", "Precision", "Recall", "F1", "RMSE"]
# table["Linear - Fold 2"] = get_metrics(best_linear)
# table["Poly - Fold 2"] = get_metrics(best_poly)
# table["RBF - Fold 2"] = get_metrics(best_rbf)

# table.head()